Google Colaboratory 환경에서 실행했습니다. (T4 고용량 RAM)

# 수정된 diffractsim 설치

In [2]:
!pip install git+https://github.com/hjw4228/diffractsim

  Cloning https://github.com/hjw4228/diffractsim to /tmp/pip-req-build-bn7sstzd
  Running command git clone --filter=blob:none --quiet https://github.com/hjw4228/diffractsim /tmp/pip-req-build-bn7sstzd
  Resolved https://github.com/hjw4228/diffractsim to commit 5b64fc7a5651f98464d1f59fcfa32b9c55b97ae7
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for diffractsim: filename=diffractsim-2.2.3-py3-none-any.whl size=71207 sha256=891880d1cadabc8697aab339ffa63aa55c77fb8ea053f538c110395ec1d03d21
  Stored in directory: /tmp/pip-ephem-wheel-cache-_vsoxtu1/wheels/7a/95/1c/3570e838dc94f93e109c381b4d628f717b2d05c03f8cbf1d8d
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12067 sha256=0c139372cf3cb5c440044b6f4edb5d1481146a26032ac0f3c39241daf2d84f45
  Stored in directory: /root/.cache/pip/wheels/cd/17/e5/765d1a3112ff3978f70223502f6047e06c43a24d7c5f8ff95b
Successfully built diffractsim progressbar


In [3]:
import gc

import diffractsim
diffractsim.set_backend("CUDA")

from diffractsim import PolychromaticField, Lens, ApertureFromImage, cf, nm, mm, cm

def render_image(f, z, mask):
  F = PolychromaticField(
    spectrum = 4 * cf.illuminant_d65,
    extent_x = 15.*mm, extent_y = 15.*mm,
    Nx=1500, Ny=1500
  )
  if mask:
    F.add(ApertureFromImage("./mask.jpg", image_size=(5. *mm, 5.* mm), simulation=F))
  else:
    F.add(ApertureFromImage("./circle.jpg", image_size=(5. *mm, 5. *mm), simulation=F))

  F.add(Lens(f=f*cm))
  F.propagate(z=z * cm)

  rgb = F.get_colors()

  if mask:
    F.plot_colors(rgb, xlim=[-6*mm, 6*mm], ylim=[-6*mm, 6*mm], save_plot=True, filename=f'{f}_{z}_masked.jpg', hide_axis=True, text='')
    print(f'\nSuccessfully rendered masked image of f={f}cm, z={F.z * 100}cm')
  else:
    F.plot_colors(rgb, xlim=[-6*mm, 6*mm], ylim=[-6*mm, 6*mm], save_plot=True, filename=f'{f}_{z}_unmasked.jpg', hide_axis=True, text='')
    print(f'\nSuccessfully rendered unmasked image of f={f}cm, z={F.z * 100}cm')\

  return

In [4]:
def render_all(start, end):
  for f in range(start, end+1):
    z = f-10
    for _ in range(1, 20):
      render_image(f, z, False)
      z += 1
    print(f'Collected garbage: {gc.collect()}')
    z = f-5

In [7]:
gc.collect()
render_all(45, 75)

Computation Took 8.659529447555542



Successfully rendered unmasked image of f=45cm, z=35.0cm


Computation Took 1.373441457748413



Successfully rendered unmasked image of f=45cm, z=36.0cm


Computation Took 1.2149336338043213


...

Computation Took 1.2935223579406738



Successfully rendered unmasked image of f=75cm, z=81.0cm


Computation Took 1.2941949367523193



Successfully rendered unmasked image of f=75cm, z=82.0cm


Computation Took 1.2939891815185547

Successfully rendered unmasked image of f=75cm, z=83.0cm
Collected garbage: 1313


In [11]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!zip data.zip *ed.jpg

updating: 45_35_unmasked.jpg (deflated 52%)
updating: 45_36_unmasked.jpg (deflated 52%)
updating: 45_37_unmasked.jpg (deflated 51%)
updating: 45_38_unmasked.jpg (deflated 51%)
updating: 45_39_unmasked.jpg (deflated 51%)
updating: 45_40_unmasked.jpg (deflated 50%)
updating: 45_41_unmasked.jpg (deflated 49%)
updating: 45_42_unmasked.jpg (deflated 49%)
updating: 45_43_unmasked.jpg (deflated 47%)
updating: 45_44_unmasked.jpg (deflated 46%)
updating: 45_45_unmasked.jpg (deflated 46%)
updating: 45_46_unmasked.jpg (deflated 45%)
updating: 45_47_unmasked.jpg (deflated 44%)
updating: 45_48_unmasked.jpg (deflated 43%)
updating: 45_49_unmasked.jpg (deflated 43%)
updating: 45_50_unmasked.jpg (deflated 41%)
updating: 45_51_unmasked.jpg (deflated 41%)
updating: 45_52_unmasked.jpg (deflated 40%)
updating: 45_53_unmasked.jpg (deflated 39%)
updating: 46_36_unmasked.jpg (deflated 51%)
updating: 46_37_unmasked.jpg (deflated 51%)
updating: 46_38_unmasked.jpg (deflated 51%)
updating: 46_39_unmasked.jpg (de